In [1]:
from IPython.core.display import Markdown, display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import traceback
import numpy as np
import scipy.stats as ss
import yaml
import itertools
import sys
import os
import subprocess
from os import path
import pickle
import pandas as pd
import matplotlib.patches as patches
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from sklearn.linear_model import LinearRegression, TheilSenRegressor, RANSACRegressor, HuberRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline

def printmd(*args):
    display(Markdown(' '.join(map(str, args))))

In [2]:
# !ros2 run local_planning_performance_modelling compute_metrics -r "~/ds/performance_modelling/output/test_local_planning/*"

In [3]:
# !ros2 run local_planning_performance_modelling compute_metrics -r "~/ds/performance_modelling/output/test_local_planning_gen_circles/*" -s

In [21]:
local_planner_color = {'dwb': 'blue', 'rpp': 'orange', 'teb': 'green'}
robot_model_color = {'hunter': 'cyan', 'turtle': 'green'}
pd.options.display.width = 500
pd.options.display.max_rows = 0
pd.options.display.float_format = '{:,.3f}'.format
plt.rcParams['figure.figsize'] = [18, 18]
plt.rcParams['lines.linewidth'] = 2
# plt.rcParams['lines.markersize'] = 3
# plt.rcParams['lines.marker'] = 'x'
plt.rcParams['scatter.marker'] = '.'

# fg_color = 'white'
# plt.rcParams['grid.color'] = 'gray'
# plt.rcParams['text.color'] = fg_color
# plt.rcParams['ytick.color'] = fg_color
# plt.rcParams['xtick.color'] = fg_color
# plt.rcParams['axes.labelcolor'] = fg_color
# plt.rcParams['axes.edgecolor'] = fg_color

# bg_color = "#323a48"
# plt.rcParams['figure.facecolor'] = bg_color
# plt.rcParams['axes.facecolor'] = bg_color
# plt.rcParams['legend.facecolor'] = bg_color

In [22]:
df_real = pd.read_csv(path.expanduser("~/ds/performance_modelling/output/test_local_planning/results.csv"))
with open(path.expanduser("~/ds/performance_modelling/output/test_local_planning/results_info.yaml")) as results_real_info_file:
    results_real_info = yaml.safe_load(results_real_info_file)
df_real = df_real[(df_real.amcl_alpha_factor.notna()) & (df_real.robot_model=='turtlebot3_waffle_performance_modelling')]
df = df_real

df.rename(inplace=True, columns={
    'localization_update_absolute_translation_error_mean': 'absolute_translation_error',
    'localization_update_absolute_rotation_error_mean': 'absolute_rotation_error',
    'localization_update_normalized_relative_translation_error_mean': 'normalized_relative_translation_error',
    'localization_update_normalized_relative_rotation_error_mean': 'normalized_relative_rotation_error',
    'localization_update_rate_mean': 'localization_update_rate',
})
df.loc[df.robot_model == 'turtlebot3_waffle_performance_modelling', 'robot_model'] = 'turtle'
df.loc[df.robot_model == 'hunter2', 'robot_model'] = 'hunter'
df['session_id'] =  df['run_id'].apply(lambda x:  x.split('_')[1]+'_'+x.split('_')[2]+'_'+x.split('_')[3])
df['run_number'] =  df['run_id'].apply(lambda x:  int(x.split('_')[5]))
df.max_steering_angle_deg = df.max_steering_angle_deg.fillna(90)

df = df[(df.max_steering_angle_deg == 90.) & (df.robot_model == 'turtle') & (df.global_planner_node == 'navfn') & (df.localization_node == 'amcl')].copy()

In [25]:
base_path = "/home/enrico/ds/performance_modelling/output/test_local_planning/"
out_dir = "plots/motions_plots/"
if not path.exists(out_dir):
    os.makedirs(out_dir)

# run_indices = [0, 1]
run_indices = df.run_index.unique()
# environment_names = ['7A-2']
environment_names = df.environment_name.unique()
local_planners = df.local_planner_node.unique()

max_vel_x = 0.26
max_vel_theta = 1.0
num_points = 1000000
for environment_name in environment_names:
    for run_index in run_indices:
        print()
        print(environment_name, run_index)
        df_s = df[(df.environment_name == environment_name) & (df.run_index == run_index)].copy()
        for local_planner in local_planners:
            run_ids = df_s[df_s.local_planner_node == local_planner].run_id
            run_output_folders = list(map(lambda x: path.join(base_path, x), run_ids))
            for i, run_output_folder in enumerate(run_output_folders):
                ground_truth_poses_file_path = path.join(run_output_folder, "benchmark_data", "ground_truth_poses.csv")
                run_events_file_path = path.join(run_output_folder, "benchmark_data", "run_events.csv")

                # get timestamps info from run events
                run_events_df = pd.read_csv(run_events_file_path, engine='python', sep=', ')
                navigation_start_events = run_events_df[run_events_df.event == 'navigation_goal_accepted']
                navigation_succeeded_events = run_events_df[(run_events_df.event == 'navigation_succeeded')]
                navigation_failed_events = run_events_df[(run_events_df.event == 'navigation_failed')]

                if len(navigation_start_events) != 1 or len(navigation_succeeded_events) + len(navigation_failed_events) != 1:
                    continue

                navigation_start_time = navigation_start_events.iloc[0].t
                navigation_end_time = navigation_succeeded_events.iloc[0].t if len(navigation_succeeded_events) == 1 else navigation_failed_events.iloc[0].t

                # get the dataframes for ground truth poses
                ground_truth_poses_df = pd.read_csv(ground_truth_poses_file_path)                
                n = int(len(ground_truth_poses_df) / num_points)
                if n < 1:
                    n = 1
                ground_truth_poses_df = ground_truth_poses_df[(navigation_start_time <= ground_truth_poses_df.t) & (ground_truth_poses_df.t <= navigation_end_time)].iloc[::n, :]

                plt.scatter(ground_truth_poses_df.v_theta, ground_truth_poses_df.v_x, marker='.', s=1**2, label=local_planner if i == 0 else None, color=local_planner_color[local_planner], alpha=0.5)#, rasterized=True)
        
        rect=patches.Rectangle((-max_vel_theta, -max_vel_x), 2*max_vel_theta, 2*max_vel_x, fill=False, color="black", linewidth=1, alpha=0.5)
        plt.gca().add_patch(rect)
        
        plt.xlabel('v_theta')
        plt.ylabel('v_x')
        plt.xlim([-2*max_vel_theta, 2*max_vel_theta])
        plt.ylim([-2*max_vel_x, 2*max_vel_x])
        plt.grid()
        plt.legend()
        plt.savefig(path.join(out_dir, f"vel_theta_x_distribution_{environment_name}_{run_index}.png"), bb_inches='tight')
#         plt.show()
        plt.close()


mexico 0

mexico 1

mexico 3

mexico 2

mexico 5

mexico 4

mexico 6

mexico 9

mexico 8

mexico 7

intel 0

intel 1

intel 3

intel 2

intel 5

intel 4

intel 6

intel 9

intel 8

intel 7

fr079 0

fr079 1

fr079 3

fr079 2

fr079 5

fr079 4

fr079 6

fr079 9

fr079 8

fr079 7

airlab 0

airlab 1

airlab 3

airlab 2

airlab 5

airlab 4

airlab 6

airlab 9

airlab 8

airlab 7

office_b 0

office_b 1

office_b 3

office_b 2

office_b 5

office_b 4

office_b 6

office_b 9

office_b 8

office_b 7

7A-2 0

7A-2 1

7A-2 3

7A-2 2

7A-2 5

7A-2 4

7A-2 6

7A-2 9

7A-2 8

7A-2 7
